In [ ]:
# -*- coding: utf-8 -*-
"""goloisConv.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1I8P7r1Y1mU3XusuUlSdjQC3nQjg_-Hba
"""

!wget https://www.lamsade.dauphine.fr/~cazenave/project.zip

!unzip project.zip
!mv ./project/* ./

!pip install pybind11
!chmod 777 ./compile.sh
!./compile.sh

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers 
from tensorflow.keras import regularizers

import golois

planes = 21   # 21 plans 19x19 les états précédents, noires, blanc
moves = 361
N = 100000
epochs = 300
batch = 200
filters = 80
#les entrées et les sorties sont fixées mais on peut modifier le réseau
input_data = np.random.randint(2, size=(N, 19, 19, planes)) #N=nombre d'exemples, pour initialise peut-être
input_data = input_data.astype ('float32')
print(input_data.shape)#added
policy = np.random.randint(moves, size=(N,))#choisi une politique aléatoire entre 1 et 361
print(policy.shape)
policy = keras.utils.to_categorical (policy)
value = np.random.randint(2, size=(N,))
value = value.astype ('float32')
print(value.shape)

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)#récupèr 10000 exemples qui n'ont pas été appris et donnés pâr le getBatch, récupèure les entrées et sortie dans la validation , c'est pour savoir si le modèle a bien appris ou pas


input = keras.Input(shape=(19, 19, planes), name='board')

x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)# on fait une convolution avec filtre 1 juste  pour  transformer le nombre de plan pour  obtenir 64 plans 


for i in range (4):#pour model résiduel
    x1 = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Conv2D(filters, 3, activation='swish',padding='same')(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Conv2D(filters, 3, activation='relu',padding='same')(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Conv2D(filters, 3, activation='relu',padding='same')(x1)

    x = layers.add([x1,x])#x is the input
    
    x = layers.ReLU()(x)
    x = layers.ReLU()(x)
    x=layers.Dropout(0.10)(x)
    x = layers.BatchNormalization()(x)

policy_head = layers.Conv2D(1, 1, activation='swish', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Conv2D(filters, 3, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)
policy_head = layers.Dense(30,activation='relu')(policy_head)
policy_head = layers.Dense(20,activation='swish')(policy_head)
policy_head = layers.Conv2D(1, 3, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)


#policy_head = layers.Conv2D(1, 3, activation='swish', padding='same')(policy_head)
#policy_head = layers.Conv2D(16, 3, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)
#policy_head = layers.Conv2D(8, 3, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)

policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)



value_head = layers.Conv2D(80, 3, activation='swish', padding='same')(x)
value_head = layers.Conv2D(60, 3, activation='relu', padding='same')(x)
value_head=layers.Dropout(0.15)(value_head)
value_head = layers.GlobalAveragePooling2D()(value_head)
value_head = layers.Dense(30, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(20, activation='swish', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head=layers.Dropout(0.10)(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0008))(value_head)




'''
value_head = layers.Conv2D(60, 3, activation='relu', padding='same')(x)
#value_head = layers.GlobalAveragePooling2D()(value_head)
value_head = layers.Dense(40, activation='swish', kernel_regularizer=regularizers.l2(0.0003))(value_head)
value_head=layers.Dropout(0.10)(value_head)
value_head = layers.Dense(30, activation='swish', kernel_regularizer=regularizers.l2(0.0003))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0008))(value_head)
'''
model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary ()


--2021-02-19 17:16:24--  https://www.lamsade.dauphine.fr/~cazenave/project.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275779208 (263M) [application/zip]
Saving to: ‘project.zip’

project.zip         100%[===================>] 263.00M  36.3MB/s    in 7.9s    

2021-02-19 17:16:33 (33.4 MB/s) - ‘project.zip’ saved [275779208/275779208]

Archive:  project.zip
  inflating: project/Board.h         
  inflating: project/compileMAC.sh   
  inflating: project/compile.sh      
  inflating: project/Game.h          
  inflating: project/games.data      
  inflating: project/golois.cpp      
  inflating: project/golois.cpython-38-x86_64-linux-gnu.so  
  inflating: project/golois.py       
  inflating: project/importGolois.ipynb  
  inflating: project/README          
  inflating: project/Rzone.h         
  infl

In [ ]:

model.compile(optimizer=keras.optimizers.SGD(lr=0.0008, momentum=0.95),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 5.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})



In [ ]:

for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups)#une partie jouée par catago , remplit les tenseurs avec tenseurs
    history = model.fit(input_data,
                        {'policy': policy, 'value': value}, 
                        epochs=1, batch_size=batch)
    if (i % 10 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)#mse sur les données de validations (su rles données qui n'ont jamais été)
        print ("val =", val)




epoch 1
500/500 [==============================] - 83s 148ms/step - loss: 9.2404 - policy_loss: 5.7579 - value_loss: 0.6948 - policy_categorical_accuracy: 0.0144 - value_mse: 0.2508
epoch 2
500/500 [==============================] - 76s 152ms/step - loss: 7.3697 - policy_loss: 3.9037 - value_loss: 0.6915 - policy_categorical_accuracy: 0.1940 - value_mse: 0.2492
epoch 3
500/500 [==============================] - 77s 154ms/step - loss: 6.7471 - policy_loss: 3.3409 - value_loss: 0.6795 - policy_categorical_accuracy: 0.2652 - value_mse: 0.2434
epoch 4
500/500 [==============================] - 77s 154ms/step - loss: 6.4970 - policy_loss: 3.1271 - value_loss: 0.6722 - policy_categorical_accuracy: 0.2946 - value_mse: 0.2399
epoch 5
500/500 [==============================] - 77s 155ms/step - loss: 6.3907 - policy_loss: 3.0327 - value_loss: 0.6698 - policy_categorical_accuracy: 0.3040 - value_mse: 0.2389
epoch 6
500/500 [==============================] - 77s 155ms/step - loss: 6.2995 - policy_

In [ ]:

model.save ('test.h5')

from google.colab import files
files.download('test.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

model.save ('test.h5')

from google.colab import files
files.download('test.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>